In [1]:
! sudo apt-get update
! sudo apt-get install -y libxrender1
! sudo apt install -y libsm6 libxext6

Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://deb.debian.org/debian-security bullseye-security InRelease [48.4 kB]
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [44.1 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8,183 kB]
Get:5 http://deb.debian.org/debian-security bullseye-security/main amd64 Packages [237 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [14.6 kB]
Fetched 8,643 kB in 3s (3,280 kB/s)




The following additional packages will be installed:
  libbsd0 libmd0 libx11-6 libx11-data libxau6 libxcb1 libxdmcp6
The following NEW packages will be installed:
  libbsd0 libmd0 libx11-6 libx11-data libxau6 libxcb1 libxdmcp6 libxrender1
0 upgraded, 8 newly installed, 0 to remove and 1 not upgraded.
Need to get 1,437 kB of archives.
After this operation, 4,065 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bullseye/main amd64 libmd0 amd64 1.0.3-3 [28.0 kB]
Get

In [156]:
import rdkit
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import Chem
import py3Dmol

In [157]:
mol = rdkit.Chem.MolFromMol2File('./simple.mol2')

[12:49:02] SIMPLE: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


In [158]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [159]:
conformers = mol.GetConformers()
C1_coords = conformers[0].GetAtomPosition(0)
C2_coords = conformers[0].GetAtomPosition(1)
C3_coords = conformers[0].GetAtomPosition(2)
C4_coords = conformers[0].GetAtomPosition(3)
print(list(C1_coords))
print(list(C2_coords))
print(list(C3_coords))
print(list(C4_coords))

[1.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[1.0, 1.0, 0.0]


In [160]:
import numpy as np
rot_mat = np.zeros((4,4))
print(rot_mat)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [183]:
#fix C1 and rotate C4
theta = np.pi/2.0
x_dash = C2_coords.x
y_dash = C2_coords.y
z_dash = C2_coords.z
dx = C3_coords.x - x_dash
dy = C3_coords.y - y_dash
dz = C3_coords.z - z_dash
l = np.sqrt(dx*dx + dy*dy + dz*dz)
s_theta = np.sin(theta)
c_theta = np.cos(theta)
l_sq = l*l


In [184]:
rot_mat[0][0] = (dx*dx + (dy*dy + dz*dz)*c_theta)/l_sq
rot_mat[0][1] = (dx*dy*(1-c_theta) - dz*l*s_theta)/l_sq
rot_mat[0][2] = (dx*dz*(1-c_theta) - dy*l*s_theta)/l_sq
rot_mat[0][3] = ((x_dash*(dy*dy + dz*dz) - dx*(y_dash*dy + z_dash*dz))*(1-c_theta) + (y_dash*dz - z_dash*dy)*l*s_theta)/l_sq

rot_mat[1][0] = (dx*dy*(1-c_theta) + dz*l*s_theta)/l_sq
rot_mat[1][1] = (dy*dy + (dx*dx + dz*dz)*c_theta)/l_sq
rot_mat[1][2] = (dy*dz*(1-c_theta) - dx*l*s_theta)/l_sq
rot_mat[1][3] = ((y_dash*(dx*dx + dz*dz) - dy*(x_dash*dx + z_dash*dz))*(1-c_theta) + (z_dash*dx - x_dash*dz)*l*s_theta)/l_sq

rot_mat[2][0] = (dx*dz*(1-c_theta) - dy*l*s_theta)/l_sq
rot_mat[2][1] = (dy*dz*(1-c_theta) + dx*l*s_theta)/l_sq
rot_mat[2][2] = (dz*dz + (dx*dx + dy*dy)*c_theta)/l_sq
rot_mat[2][3] = ((z_dash*(dx*dx + dy*dy) - dz*(x_dash*dx + y_dash*dy))*(1-c_theta) + (x_dash*dy - y_dash*dx)*l*s_theta)/l_sq

rot_mat[3][0] = 0.0
rot_mat[3][1] = 0.0
rot_mat[3][2] = 0.0
rot_mat[3][3] = 1.0

print(rot_mat)

[[ 6.123234e-17  0.000000e+00 -1.000000e+00  0.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]


In [185]:
C4_vector = list(C4_coords)
C4_vector.append(1.0)
C4_array = np.array(C4_vector).reshape((4,1))
C4_array

array([[1.],
       [1.],
       [0.],
       [1.]])

In [186]:
np.matmul(rot_mat, C4_array)


array([[ 6.123234e-17],
       [ 1.000000e+00],
       [-1.000000e+00],
       [ 1.000000e+00]])